# Similarity Collaborative Recommenders

## Objectives

* Explain how similarity-based collaborative recommenders differ from content-based recommenders.
* Discuss the differences between and relative advantages of item-item and user-user similarity recommenders.
* Explain how and why neighborhoods are calculated for similarity recommenders.
* Discuss two problems with the evaluation of recommenders

## Agenda:  
 1. Terminology for recommenders  
 2. Data format for collaborative recommenders
     Explicit vs Implicit ratings
 3. Similarity collaborative recommendations
    User-User Method vs Item-Item Method
 4. Pros and cons of collaborative filtering (similarity collaborative filters)
 5. Issues with evaluation of recommenders  

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## Introduction

Recap:  
 - **Content recommenders** only take into account the item features and the given user actions/ratings.   
 - **Collaborative recommenders** take into account the actions of other users to make recommendation.
 

Collaborative recommenders can be built to give more diverse recommendations there is more vocabulary we need to talk about.

Collaborative recommenders can be based on:
* memory (i.e. past user behavior / ratings)
* models - SVD and NMF to discover latent factors

### Terminology 

 - **Serendipity**: How surprising are the recommendations? Did you recommend something the user wouldn’t have otherwise seen? (If you ask your best friend for a single book recommendation of a really good book and they say Harry Potter, they maybe kinda blew it (because you already knew that))  
 - **Personalization**: Populist versus personalized. Popular can seem dumb; too personal can be creepy. And popular doesn’t always work.  
 - **Diversity**: Did you recommend Harry Potter 1, 2, 3, ...? “Silver Bullet” vs Shotgun. But also want basket approach (beer with those diapers?). Breadth vs Depth. Breadth and Depth?
 - **Persistence**: Have you recommended this item before?
 - **Modality**: I usually want Sweet Tomatoes, but tonight I’m in a different modality because it’s date night and my wife doesn’t like Sweet Tomatoes
 - **Privacy**: You’re generally pretty similar to Moses, and he just bought furry pink handcuffs, do you want some?
 - **Motivation**: Pure best match? (Medical Treatment) Overall product value? (Pandora) Specific sale/margin? (Amazon) Upsell? (Wells Fargo)
 - **Trust**: What is the motivation? What data is offered, used? How is it being used? Why am I seeing what I’m seeing?
 - **Confidence**: How good are the results? How understandable are the mistakes? Harry Potter 7 is an understandable mistake, 50 Hairdos for Terrible People might not be. Confidence over time, especially: building trust first and then branching out. 

##  Collaborative Types of Data

Data can be:

*   *Explicit*:
    -   User provided ratings (1 to 5 stars)
    -   User like/non-like
*   *Implicit*:
    -   Infer user-item relationships from behavior
    -   More common
    -   Example: buy/not-buy; view/not-view

<br/><br/>

<details><summary><font size='3'>
    <italics> Q: What are some other ways of capturing implicit data? </italics> 
    </font>
</summary>
    
<font size='3'>
 - Time viewed (YouTub, Netflix)    <br/>
 - Number of times clicked/listened (Pandora) <br/>  
 - Percentage scrolled (NY Times) <br/>  
    <br/>
        <br/>
 One thing to take away is number of times listened to or watched has become a powerful method of not just creating recommendations but also to evaluate a new one.
    
</details>
<br/><br/>

### User Data

Recommender systems are a subclass of information filtering.  The idea is finding methods to filter the huge amount of data/options we have over the internet into what the user is interested in.  With this in mind being able to segment a population into similar groupings to may be useful for recommendations.   

<br/><br/><br/>

<details><summary><font size='4'>
Q: What might be some useful things to know about a user.  What may we have access to? 
    </font>
</summary>
    
<font size='3'>
    <br/>
If they signed in we have a user profile and past actions.  At the least we have IP address for location and the use of cookies.  There is also how they ended up at the site (ie google, or another site).
    
</details>
<br/><br/><br/>
    
We call the matrix of user-item data a **utility matrix**. 
**The goal of collaborative filtering is to complete this utility matrix in order to make recommendations**.

####  Example:    Explicit utility matrix

Example 9.1 in [Mining of Massive Datasets](http://infolab.stanford.edu/~ullman/mmds/book.pdf):

| &nbsp;| Item1   | Item2   | Item3   | Item4    | Item5   | Item6   | Item7   |
| :---- | :---- | :--   | :--   | :--   | :--   | :--   | :--   |
| User1     |  4    | &nbsp;| &nbsp;| 5     | 1     | &nbsp;| &nbsp;|
| User2     |  5    | 5     | 4     | &nbsp;| &nbsp;| &nbsp;| &nbsp;|
| User3     | &nbsp;| &nbsp;| &nbsp;| 2     | 4     | 5     | &nbsp;|
| User4     | &nbsp;| 3     | &nbsp;| 5     | 1     | &nbsp;| 3     |

---

####  Example:    Implicit utility matrix


| &nbsp;| Item1   | Item2   | Item3   | Item4    | Item5   | Item6   | Item7   |
| :---- | :---- | :--   | :--   | :--   | :--   | :--   | :--   |
| User1     |  1    | &nbsp;| &nbsp;| 1     | 1     | &nbsp;| &nbsp;|
| User2    |  1    | 1     | 1     | &nbsp;| &nbsp;| &nbsp;| &nbsp;|
| User3     | &nbsp;| &nbsp;| &nbsp;| 1     | 1     | 1     | &nbsp;|
| User4     | &nbsp;| 1     | &nbsp;| 1     | 1     | &nbsp;| 1     |

### Prediction vs Ranking

There are 2 possible outcomes when talking about recommenders.  With explicit recommenders there is the goal of **predicting** the rating a user may give a item.  The other outcome is **ranking** based.  That is showing $n$ items the user will want to interact with.  

*How would we go about testing these predictions?*

# Collaborative Similarity Recommenders
<br/>  
 
There are 2 sub-types of similarity recommenders:
 - User-User Similarity
   * Comparing user preferences to other users
   * Idea: Similar users give similar ratings to a single product
   * Generally a specific user only interacts with a small subset of products, so can get high-variance, low bias estimates
   
   
 - Item-Item Similarity
   * Comparing items to items
   * Idea: A single user will give similar ratings to similar products
   * Many users interact with a specific product, so item ratings are robust, so can get low variance, high bias estimates
   
   

| &nbsp;| HP1   | HP2   | HP3   | TW    | SW1   | SW2   | SW3   |
| :---- | :---- | :--   | :--   | :--   | :--   | :--   | :--   |
| Al     |  4    | &nbsp;| &nbsp;| 5     | 1     | &nbsp;| &nbsp;|
| Bob     |  5    | 5     | 4     | &nbsp;| &nbsp;| &nbsp;| &nbsp;|
| Cat     | &nbsp;| &nbsp;| &nbsp;| 2     | 4     | 5     | &nbsp;|
| Dan     | &nbsp;| 3     | &nbsp;| 5     | 1     | &nbsp;| 3     |



When talking about the utility matrix let:
 - $m$ be number of users
 - $n$ be the number of items 





**Question**: If there are $m$ users and $n$ items, we need to compute the similarity of all pairs; thus the preprocessing time complexity for user-user similarity is $\mathcal O(m^2n)$ (how so?) and item-item similarity is $\mathcal O(mn^2)$, with respective space complexity estimates of $\mathcal O(m^2)$ and $\mathcal O(n^2)$. Which one is more algorithmicly efficient?

Item-Item is more common form of recommender (originally it was user-user but Amazon showed the effectiveness of item-item).  For the rest of the talk I will be using item-item system, to change to user-user you could simply flip the matrix first

## Creating Predictions

Say user $u$ hasn't rated item $i$.  We want to predict the rating this user would give the item.    


Approach:
- Get utility matrix (user-item rating matrix)
- Get item-item (or user-user) similarity matrix
- Based on user's ratings of items and these items' similarities to the predicted item, calculate rating

### Similarity

$$
rating(u,i) = \frac{ \sum_{j \in I_u} similarity(i,j) * r_{u,j} } {\sum_{j  \in I_u} similarity(i,j) }
$$    
   
Where:  
 - $I_u$ = set of items rated by user $u$
 - $r_{u,j}$ = user $u$'s rating of item $j$
 - $sim(i, j)$ =  similarity between items i and j
 
 
 *What kind of similarity metrics can be used?*

<img src="../images/item_item_example1.png" alt="Item-Item utility matrix">
<img src="../images/item_item_example2.png" alt="Item-Item utility matrix">

<br/>

<details><summary><font size='4'>
Q: Is there something that can be done to speed this up? 
    </font>
</summary>
    
<font size='3'>
    <br/>
Should we be calculating a user's rating for items that are not similar?
    
</details>
<br/><br/><br/>

## Using Neighborhoods

To speed up the calculation we can just look at the $n$ most similar items when calculating the prediction.   

Updated approach:
- Get utility matrix (user-item rating matrix)
- Get item-item (or user-user) similarity matrix
- Order user's rated items by similarity
- Select the neighborhood (i.e. most similar items)
- Based on user's ratings of neighboring items and these items' similarities to the predicted item, calculate rating
   
$$
rating(u,i) = \frac{ \sum_{j \in I_u \cap N_i} similarity(i,j) * r_{u,j} } {\sum_{j  \in I_u \cap N_i} similarity(i,j) }
$$     

Where:  
 - $I_u$ = set of items rated by user $u$
 - $r_{u,j}$ = user $u$'s rating of item $j$
 - $N_i$ = $n$ most similar items to $i$   
    


In [2]:
# In the example above, how would Cat rate B, using the top 2 most similar items?

##  Recommend best items

Recommend items with highest predicted rating:

*   Sort predicted ratings $\hat{r}_{ui}$
*   Optimize by only searching a neighborhood which contains the $n$ items most similar to $i$
*   Beware:
    -   Consumers like variety
    -   Don't recommend every Star Trek film to someone who liked first film
    -   Best to offer several different types of item

## Deploying a Recommender

 - Compute the similarity matrix and user/item neighborhoods off line.  
 - Compute predicted ratings/rankings live

What should you think about when you hear neighborhoods? 


## Advantages and Disadvantages of Similarity-Based Collaborative Filtering

### Advantages
- Serendipitous recommendations
- No domain knowledge / feature extraction necessary

### Disadvantages
- Cannot predict ratings for new users or new items
- Computationally expensive!
- Scalability
- Sparsity: Too little data on item(s) to make good prediction
- Hard to include item features



##  The cold-start problem

It's difficult to build a recommender without ratings!
*   Must also handle new users and new items

*   *Cold-start* problem:
    -   Need utility matrix to recommend!

*   Approaches:
    -   Can ask new users to rate items
    -   Infer ratings from behavior, e.g., viewing an item
    -   Use ensemble of (bad) recommenders until you have enough ratings
    -   Use content-based recommender
    -   Exploit implicit, tag, and other side data
    -   Use `ItemSimilarityModel` until you have enough rating data
    

# Evaluating a Recommender

Evaluation of recommenders is a difficult task with many things to keep in mind.  There is not a **best** method and the methods should be based on the business goal.  There are many things to keep in mind when putting together a evaluation metric and many times these are custom to the problem.  Things to think about:

 - Given the utility matrix how would split data to evaluate your recommender?  
 - Can we use the same test train split method?  
 - What issues may we have splitting temporally?
 - RMSE and MAE are historically used (Netflix prize): what are the drawbacks?
 - Is there equal weight in getting low ratings wrong as high?
 
 
That being said the best way to validate a recommender is A/B test in the real world as usually lift or hit rate are the real business goals.  

## Ranking Metrics

Ranking metrics are useful to test how well a recommender is doing as many times predicting the score a user would give is not important, showing something they will click is.  Below is a method of scoring a implicit recommender where they have number of interactions.  This is originally from [this paper](http://yifanhu.net/PUB/cf.pdf) for a music recommender.

$$
\overline{rank} = \frac{\sum_{u,i} r^t_{ui} * rank_{ui}}{\sum_{u,i} r^t_{ui}}
$$
Where:  
 - $r^t_{ui}$ is the # of interactions or observations in the test set.
 - $rank_{ui}$ are the percentile ranking of each item for each user in reverse order.
    
Here more weight is being placed on order of selecting of the items.  The most used items should be predicted highest and thus have a $rank_{ui}$ close to $0$ while low interaction things should have a high $rank_{ui}$.  Thus means the lower the score the better the recommendations

When working with recommenders you should be ready to spend significant time deciding the best way to validate your results. Another large part of the success of recommenders is based on UX design and how it is introduced to the user. It has been found that many times the **best** recommender is less well received than other options

# Review
 - Types of Data
   * Explicit 
   * Implicit
 - Similarity Metrics 
   * Euclidean
   * Pearson
   * Cosine Similarity
   * Jaccard
 - What are the steps to an Item-Item collaborative Nearest Neighbor Similarity Recommender?